In [1]:
import glob
import pandas as pd
import os

import pyspark
import warnings
warnings.filterwarnings(action='ignore')


from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

In [2]:
agg_df = spark.read.parquet('agg_data_y_m.parquet')

In [3]:
agg_df.count()

1364775

In [5]:
df_pd = agg_df.toPandas()

In [6]:
df_pd.head(10)

,YEAR,MONTH,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LONG,MEAN_LAT
0,2015,01,AK HOONAH,213.0,0.0,508.0,0.0,13.516129,13.516129,0.0,61.0,-135.42780,58.107200
1,2015,01,AL DECATUR,0.0,0.0,467.0,0.0,0.000000,0.000000,0.0,0.0,-86.93814,34.578467
2,2015,01,AL VERNON,0.0,0.0,445.0,10.0,0.000000,0.000000,0.0,56.0,-88.09260,33.701600
3,2015,01,AZ SNOWFLAKE,81.0,0.0,107.0,0.0,3.419355,3.419355,0.0,0.0,-110.16680,34.496900
4,2015,01,AZ Snowslide Canyon,0.0,0.0,229.0,0.0,0.000000,0.000000,0.0,0.0,-111.65000,35.340000
5,2015,01,CA BOONVILLE,0.0,0.0,86.0,0.0,0.000000,0.000000,0.0,0.0,-123.40840,39.006200
6,2015,01,CA FRIEND MOUNTAIN CALIFORNIA,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,-123.34170,40.505000
7,2015,01,CA WOODSIDE,0.0,0.0,51.0,0.0,0.000000,0.000000,0.0,0.0,-122.26370,37.375400
8,2015,01,CO GALETON,13.0,0.0,23.0,0.0,0.838710,0.838710,0.0,0.0,-104.56470,40.503800
9,2015,01,CO HARTSEL,64.0,0.0,36.0,0.0,3.200000,3.200000,0.0,0.0,-105.77500,38.846800


In [9]:
df_pd.to_csv('weather_data_year_month.csv')

In [14]:
lat_long_details = df_pd[['STATE_CITY_NAME','MEAN_LONG','MEAN_LAT']]

In [32]:
lat_long_details['STATE_CITY_NAME'].nunique()

20677

In [22]:
year_level= df_pd.groupby(by=['YEAR','STATE_CITY_NAME'],as_index=False).agg({'MAX_SNOW':'max',\
                                                                             'MIN_SNOW':'min',\
                                                                             'MAX_PRCP':'max',\
                                                                             'MIN_PRCP':'min',\
                                                                             'MEAN_SNOW':'mean',\
                                                                             'MEAN_PRCP':'mean',\
                                                                             'MEDIAN_SNOW':'median',\
                                                                             'MEDIAN_PRCP':'median',\
                                                                              'MEAN_LAT':'mean',\
                                                                            'MEAN_LONG':'mean',})

In [27]:
year_level.to_csv('agg_year.csv',index=False)

In [34]:
year_level.groupby(['STATE_CITY_NAME'],as_index=False).count()['YEAR']

,STATE_CITY_NAME,YEAR,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
0,AK ALCAN HWY MI-,7,7,7,7,7,7,7,7,7,7,7
1,AK ALPINE,7,7,7,7,7,7,7,7,7,7,7
2,AK ALPINE CREEK LODGE,3,3,3,3,3,3,3,3,3,3,3
3,AK ALYESKA,7,7,7,7,7,7,7,7,7,7,7
4,AK AMBER LAKE,7,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
20672,WY YELLOWSTONE LAKE,7,7,7,7,7,7,7,7,7,7,7
20673,WY YELLOWSTONE NP E ENT,7,7,7,7,7,7,7,7,7,7,7
20674,WY YELLOWSTONE PARK MAMMOTH HCN,7,7,7,7,7,7,7,7,7,7,7
20675,WY YODER,7,7,7,7,7,7,7,7,7,7,7


In [36]:
year_level.loc[year_level['STATE_CITY_NAME']=='AK ALPINE CREEK LODGE']

,YEAR,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
2,2015,AK ALPINE CREEK LODGE,201.0,0.0,617.0,0.0,5.302560,5.302560,0.0,0.0,63.0428,-147.2478
17387,2016,AK ALPINE CREEK LODGE,305.0,0.0,838.0,0.0,2.608154,2.608154,0.0,0.0,63.0428,-147.2478
34884,2017,AK ALPINE CREEK LODGE,305.0,0.0,89.0,0.0,5.093030,5.093030,0.0,0.0,63.0428,-147.2478


In [37]:
year_level.loc[year_level['STATE_CITY_NAME']=='AK ALPINE']

,YEAR,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
1,2015,AK ALPINE,241.0,0.0,254.0,0.0,3.009594,3.009594,0.0,0.0,70.3464,-150.9283
17386,2016,AK ALPINE,46.0,0.0,226.0,0.0,1.254313,1.254313,0.0,0.0,70.3464,-150.9283
34883,2017,AK ALPINE,66.0,0.0,191.0,0.0,1.829781,1.829781,0.0,0.0,70.3464,-150.9283
52446,2018,AK ALPINE,76.0,0.0,152.0,0.0,1.508826,1.508826,0.0,0.0,70.3464,-150.9283
70044,2019,AK ALPINE,51.0,0.0,119.0,0.0,1.160721,1.160721,0.0,0.0,70.3464,-150.9283
87570,2020,AK ALPINE,58.0,0.0,130.0,0.0,1.181124,1.181124,0.0,0.0,70.3464,-150.9283
105259,2021,AK ALPINE,64.0,0.0,86.0,0.0,1.715591,1.715591,0.0,0.0,70.3464,-150.9283
